(assignment_5)=

# Assignment 5: Global overturning circulation


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

def subplots_shared(n, m, sharex=True, sharey=True, layout='constrained', figsize=None):
    return plt.subplots(n, m, sharex=sharex, sharey=sharey, layout=layout, figsize=figsize)

def pcolormesh_sym(x, y, z, vbound, ax=None, cmap='RdBu_r'):
    if ax is None:
        ax = plt.gca()
    return ax.pcolormesh(x, y, z, vmin=-vbound, vmax=vbound, cmap=cmap, rasterized=True)

def tohours(Time):
    return Time.values.astype('float') / 1e9 /3600



In [ ]:
%matplotlib widget

## Q1.1 Investigate the structure of the deep water circulation

### Q1.1.1 Plot a cross section of north/south velocity

Plot an east-west section of north-south velocity in one of the runs (I used `OverturningBeta10e5Coarse.snapshot.nc`) close to the middle of the basin (say 30 or 40 N).  Comment on the flow patterns, both in the veritcal and across the basin.  


In [ ]:
with xr.open_dataset('../Assign4/OverturningBeta10e5Coarse.snapshot.nc') as ds:
    lat0 = 30.0
    ds = ds.isel(Time=-1).sel(yt=lat0, yu=lat0, method='nearest')
    fig, axs = plt.subplots(layout='constrained')
    pc = pcolormesh_sym(ds.xu, ds.zt, ds.v, vbound=0.01, ax=axs)
    axs.contour(ds.xu, ds.zt, ds.temp, colors='k', levels=np.arange(-1, 20, 1), linewidths=0.5)
    axs.set_ylabel('Depth [m]')
    axs.set_xlabel('Longitude [deg]')
    fig.colorbar(pc, ax=axs, extend='both', shrink=0.6)
    axs.set_title(f'North-south velocity [m/s] at {lat0} degrees N')

There is clearly a two-layer flow, with opposing layers above and below about 1000 m. In the deep layer there is a strong southwards flow on the western boundary coupled with a weaker northwards flow in the interior.  

## Q1.1.2 Quantify the north-south transport

For the whole basin, consider the deeper layer you identified above and calculate the vertically integrated north south transport, per degree of longitude, in the basin and plot.  Do the same for the east-west transport, and comment on the structures.

In [ ]:
with xr.open_dataset('../Assign4/OverturningBeta10e5Coarse.snapshot.nc') as ds:
    ds = ds.isel(Time=-1)
    sumv = ds.v.sel(zt=slice(-10_000, -1_200)).integrate(coord='zt')
    sumu = ds.u.sel(zt=slice(-10_000, -1_200)).integrate(coord='zt')
    fig, axs = plt.subplots(1, 2, sharex=True, sharey=True, layout='constrained')
    pcolormesh_sym(ds.xt, ds.yu, sumv, 30, ax=axs[0])
    axs[0].set_title('North-south transport \nper degree longitude [m^2/s]', fontsize='medium')
    axs[0].set_ylabel('Lat [degrees]')
    pc = pcolormesh_sym(ds.xt, ds.yu, sumu, 30, ax=axs[1])
    axs[1].set_title('East-west transport \nper degree latitude [m^2/s]', fontsize='medium')
    axs[1].set_xlabel('Lon [degrees]')
    fig.colorbar(pc, ax=axs, extend='both', shrink=0.6)

The western boundary current is clear in this plot.  The return flow in the interior is not completely symmetric, with the return flow concentrated towards the western side.  This indicates that the Stommel and Arons assumption of uniform upwelling in the interior may not apply here.  Note that the east-west transport supplies water to flow north in the interior, and then much of the flow is to the west in the norht part of the basin.

## Q1.1.3 Plot w

Plot w at the top of the volume you considered in the previous question and discuss if the Stomel-Arons assumption of uniform w is applicable.

In [ ]:
with xr.open_dataset('../Assign4/OverturningBeta10e5Coarse.snapshot.nc') as ds:
    ds = ds.isel(Time=slice(-10, -1)).mean(dim='Time')
    fig, axs = plt.subplots()
    pc = pcolormesh_sym(ds.xt, ds.yt, ds.w.sel(zw=-1000, method='nearest'), 2e-6)
    fig.colorbar(pc, ax=axs, shrink=0.6, extend='both')
    axs.set_title('w at 1000 m [m/s]')
    axs.set_xlabel('Lon [deg]')
    axs.set_ylabel('Lat [deg]')

So we see that w is quit inhomogenous, with stronger w where we see stronger northward flow.  So Sverdrup dynamics is likely OK, but the assumption of uniform upwelling is not correct here.  Note there are also strong vertical velocities at the boundaries, but they are not the focus.

## Q1.1.4 Transport as function of latitude

Choose a central latitude, and plot the total transport in the deep layer as a function of longitude.  Comment on the net transport across this section compared to the peak transport in the section, with reference to the expectations from the Stomel-Arons theory.


In [ ]:
with xr.open_dataset('../Assign4/OverturningBeta10e5Coarse.snapshot.nc') as ds:
    ds = ds.isel(Time=-1)
    sumv = ds.v.sel(zt=slice(-10_000, -1_200)).integrate(coord='zt')

    fig, ax = plt.subplots(2, 1, sharex=True, layout='constrained')
    lat0=40
    ax[0].plot(sumv.xt, sumv.sel(yu=lat0, method='nearest'))
    ax[0].set_ylabel('Transport bottom \n layer[m^2/s]')
    ax[0].axhline(0, linestyle='--', color='k')
    dx = 4 * 110e3 * np.cos(np.pi*lat0/180)
    ax[1].plot(sumv.xt, sumv.sel(yu=lat0, method='nearest').cumsum() * dx/ 1e6)
    ax[1].set_ylim([-20, 0])
    ax[1].set_ylabel('Cumulative Transport \n bottom layer [Sv]')


So the net transport is about 10 Sv to the south in the deep layer.  The peak transport going south is almost twice as large, so the deep western boundary current is strengthened by the gyre set up by the Sverdrup-like dynamics due to the upwelling of water in the interior.  

# Q1.2 Available potential energy

Lets examine the _idea_ of available potential energy in these simulations.  We won't properly calculate available potential energy because that is quite difficult in the spehrical geometry we ran these simulations with.  Instead, lets take one central longitude, and pretend that the longitude dimension has the same `dx` for each latitude (as opposed to in the actual spherical geometry, where dx gets narrower to the north).  

# Q1.2.1 Calculate the background potential energy

for each of 

```
todo = ['../Assign4/OverturningBeta4e6Coarse.snapshot.nc', 
        '../Assign4/OverturningBeta1e5Coarse.snapshot.nc', 
        '../Assign4/OverturningBeta4e5Coarse.snapshot.nc',
        '../Assign4/OverturningBeta8e5Coarse.snapshot.nc',
        '../Assign4/OverturningBeta10e5Coarse.snapshot.nc',
        '../Assign4/OverturningBeta4e4Coarse.snapshot.nc']
```

calculate a sorted version of the density (`ds.prho`) in a section at the central latitude, with the densest water at the bottom and lightest water at the surface.  

This is a bit tricky, because the depth bins are not even!  So first I selected my time and values of `xt` and made a new data slice interpolated onto a new grid:

```
dsnew = ds.sel(zt=np.arange(-4000, 0, 20), method='nearest')
```

Then I used `np.sort` and `np.reshape` to make a new `prho0`.

To show that you did this properly, contour `prho` and `prho0` for one of the simulations, using the same density contours.


In [ ]:
td = '../Assign4/OverturningBeta10e5Coarse.snapshot.nc'
with xr.open_dataset(td) as ds:
    dsnew = ds.isel(Time=-1).isel(xt=10, xu=10).sel(zt=np.arange(-4000, 0, 20), method='nearest')
    dsnew['dzt'] = 20
    dsnew['prho0'] = (('zt', 'yt'), 
                          np.reshape(np.sort(dsnew.prho.values.flat)[::-1], 
                                     [dsnew.dims['zt'], dsnew.dims['yt']]))
    fig, ax = plt.subplots()
    levels = np.arange(-2, 2, 0.25)
    ax.contour(dsnew.yt, dsnew.zt, dsnew.prho, levels=levels)
    pp = ax.contour(dsnew.yt, dsnew.zt, dsnew.prho0, levels=levels, linestyles='--')
    fig.colorbar(pp)
    ax.set_ylabel('Depth [m]')
    ax.set_xlabel('Lat [deg]')
    ax.set_title(td)

    print(np.mean(dsnew.prho0))
    print(np.mean(dsnew.prho))

We can see that the sorted version is not exactly flat - there are some isopycnals that have a jump because the sorting still has some inhomogenity at each depth.  We could average across latitudes to remove this, but it should not change the outcome substantially.

## Q1.2.2 Calculate the Available Potential Energy 

Calculate the PE of the background state and the actual state and calculate the APE for each of the simulations.  Make a log-log plot of K versus Background potential energy and a second of K versus APE.  Note that BPE can be negative, depending on what reference z you choose, so try adding an offset to z to make the BPE positive.  Note that APE should _not_ be affected by your choice of the z offset (and indeed changing the offset is a very useful way to demonstrate that you did the calculation correctly).  Express your ponential energy in units of J/m.  

Comment on the trend in background potential energy as a function of K, and the available potential energy as functions of K. 

In [ ]:
PE = np.zeros(6)
PE0 = np.zeros(6)
todo = ['../Assign4/OverturningBeta4e6Coarse.snapshot.nc', 
        '../Assign4/OverturningBeta1e5Coarse.snapshot.nc', '../Assign4/OverturningBeta4e5Coarse.snapshot.nc',
        '../Assign4/OverturningBeta8e5Coarse.snapshot.nc',
        '../Assign4/OverturningBeta10e5Coarse.snapshot.nc',
        '../Assign4/OverturningBeta4e4Coarse.snapshot.nc',
]

K = [4e-6, 1e-5, 4e-5, 8e-5, 1e-4, 4e-4]
for nn, td in enumerate(todo):
    with xr.open_dataset(td) as ds:
        
        dsnew = ds.isel(Time=-1).isel(xt=10, xu=10).sel(zt=np.arange(-4000, 0, 20), method='nearest')
        dsnew['dzt'] = 20
        zoffset = 4000
        PE[nn] = ((dsnew.prho) * (dsnew.zt+zoffset) * dsnew.dzt * dsnew.dyt * 4 * 111e3).sum().values
        dsnew['rho0'] = (('zt', 'yt'), 
                      np.reshape(np.sort(dsnew.prho.values.flat)[::-1], [dsnew.dims['zt'], dsnew.dims['yt']]))

        PE0[nn] = ((dsnew.rho0) * (dsnew.zt+zoffset) * dsnew.dzt * dsnew.dyt* 4 * 111e3).sum().values


In [ ]:
fig, ax = plt.subplots(2, 1, layout='constrained')
ax[0].loglog(K, PE, 'd')
ax[0].loglog(K, PE0, 'd')
ax[0].set_ylabel('BPE [J/m] zoffset = 4000 m')
ax[1].loglog(K, PE-PE0, 'd')
ax[1].set_xlabel('K [m^2/s]')
ax[1].set_ylabel('APE [J/m]')

Background potential energy drops as K is increased. This is because the ocean holds more heat with more mixing than if there is less mixing.  

Available potential energy increases with K because the buoyancy input near the south end of the basin is mixed down, and creates stron isopycnal slopes.  These of course create pressure gradients that subsquently drive the overturning circulation.